In [ ]:
# 11
# ENST00000267116.8-chr12:56239880-56239973
# hg38
# mammal
# mammal
# 0
# 0
# 3
# 0
# VFLGVEGLRRAYLPLQPQPWPLFLPHPYPF
# 17
# ENST00000307714.12+chr1:32036957-32038075
# hg38
# mammal
# mammal
# 0
# 0
# 3
# 1
# LNGVPEPSRGRGVPVRGRGAAPPPPPVPRGRGVGPPRGALVRGTPV

In [1]:
tree = "/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/results_120/prank/ENST00000267116.8-chr12__56239880-56239973.maf.best.anc.dnd" #prot tree best.anc.dnd

In [9]:
tree = "/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/results_120/prank/ENST00000307714.12+chr1__32036957-32038075.maf.best.anc.dnd" #prot tree best.anc.dnd

In [10]:
import sys
import subprocess
import os
import glob
from Bio import SeqIO
from Bio.Seq import Seq
import os.path

# assign each species to the lineage
order = {}
for sp in ("panTro5","panPan2","gorGor5","ponAbe2","nomLeu3"):
	order[sp] = "hominoid"
for sp in ("rheMac8","macFas5","macNem1","papAnu3","manLeu1","cerAty1","chlSab2","nasLar1","rhiRox1","rhiBie1","colAng1","HLpilTep1"):
	order[sp] = "catarrhini"
for sp in ("calJac3","aotNan1","saiBol1","cebCap1"):
	order[sp] = "simiiformes"
for sp in ("tarSyr2","otoGar3","micMur3","proCoq1"):
	order[sp] = "primates"
for sp in ("galVar1",):
	order[sp] = "primatomorpha"	
for sp in ("tupChi1","jacJac1","micOch1","criGri1","mesAur1","perManBai1","mm10","HLmusCar1","HLmusPah1","rn6","HLmerUng1","nanGal1","HLcasCan1","HLdipOrd2","hetGla2","HLfukDam1","cavPor3","chiLan1","octDeg1","speTri2","HLmarMar1","oryCun2","ochPri3"): #rodents+primates
	order[sp] = "euarchontoglires"

cetartiodactyla =("vicPac2","HLcamFer2","HLcamBac1","HLcamDro1","HLturTru3","HLorcOrc1","HLdelLeu1","lipVex1","phyCat1","balAcu1","HLbalMys1","bosTau8","HLbosInd1","bisBis1","bosMut1","bubBub1","HLoviAri4","HLoviCan1","HLcapHir2","panHod1","HLodoVir1","HLcerEla1","susScr11")
perissodactyla = ("HLequCab3","equPrz1","HLequAsi1","cerSim1")
ferae = ("felCat8","HLaciJub1","panTig1","HLpanPar1","canFam3","HLlycPic1","musFur1","HLenhLut1","HLailFul1","ailMel1","ursMar1","odoRosDiv1","lepWed1","neoSch1","manPen1","HLmanJav1")
chiroptera = ("pteAle1","HLpteVam2","rouAeg1","HLrhiSin1","HLhipArm1","eptFus1","myoDav1","myoBra1","myoLuc2","HLminNat1","HLdesRot1")
eulipotyphla = ("eriEur2","sorAra2","conCri1")
boreoeutheria = cetartiodactyla + perissodactyla + ferae + chiroptera + eulipotyphla

for sp in boreoeutheria: #previous+euarchontoglires
	order[sp] = "boreoeutheria"
for sp in ("loxAfr3","triMan1","HLproCap2","chrAsi1","echTel2","eleEdw1","oryAfe1","dasNov3","HLchoHof2"): #atlantogenata +boro
	order[sp] = "placentalia"
for sp in ("monDom5","sarHar1","HLphaCin1","ornAna2"): #nonplacental+rest
	order[sp] = "mammal"

# main
fastan = tree.replace(".dnd",".fas")
name = tree.split("/")[-1].replace(".maf.best.anc.dnd","")
tag = tree.replace(".maf.best.anc.dnd","")

# generate ancestral protein
fastap = open(tree.replace(".dnd",".prot.fas"),"w+")
fa = {}
fan = SeqIO.index(fastan, "fasta")
for n in fan:
	fa[n] = str(Seq(str(fan[n].seq).replace("-","")).translate(cds = False))
	fastap.write(">" + n + "\n" + fa[n] + "\n")
fastap.close()

fline=open(tree).readline().rstrip().split(")")
last_sp = "hg38"
sub = 0
ancestor ={}
for n,line in enumerate(fline):
	if sub != 0:
		sub = sub - 1
		if not last_sp in ancestor:
			ancestor[last_sp] = ["none",[]]
		ancestor[last_sp][1].append(line.split(":")[0])
	else:
		if last_sp != "hg38":
			if not last_sp in ancestor:
				ancestor[last_sp] = ["none",[]]
			ancestor[last_sp][0] = line.split(":")[0]
		try:
			last_sp = line.split(",")[1].split(":")[0].replace("(","")
		except:
			pass
	if n != 0:
		sub = sub + line.count("(")
# print species and ancestor tree
print(ancestor)

prot_ancestors_file = tag + ".prot.ancestors"
ancestors_file = tag + ".ancestors"
nucl_ancestors_file = tag + ".nucl.ancestors"
# 检查并删除已存在的文件
if os.path.exists(prot_ancestors_file):
    os.remove(prot_ancestors_file)
if os.path.exists(ancestors_file):
    os.remove(ancestors_file)
if os.path.exists(nucl_ancestors_file):
    os.remove(nucl_ancestors_file)
# 打开文件进行写入
outp = open(prot_ancestors_file, "a+")
outh = open(ancestors_file, "a+")
outn = open(nucl_ancestors_file, "a+")
# 初始化变量
max_st_cons = "human"
max_sy_cons = "human"
outp.write('orf_id\tsp\tev_age\tsyn_age\tbranch\tTIS\tmaxORF\tconv\tseq\n')
outh.write('orf_id\tsp\tev_age\tsyn_age\tgained\tgained_convergent\tlost\tdenovo\tseq\n')

# ORF structure
## 进化上是否存在
ev = {}
## 最大的重叠比例
max_op ={}
## 起始密码子
tis={}
al_scodon = ["ATG", "CTG", "ACG", "GTG", "TTG", "ATA", "ATC", "ATT", "AAG", "AGG"]

# estimate the presence of intact ORF
for last_sp in ancestor:
    tis[last_sp] = "0"
    old_anc = ancestor[last_sp][0]
    if len(str(fa[old_anc]).replace("-","").replace("X","*")) >= 4:
        # 看是否在第一个位置或者下游的两个位置有非经典的起始密码子
        first_scodon=str(fan[old_anc].seq).replace("-","")[0:3].upper()
        second_codon=str(fan[old_anc].seq).replace("-","")[3:6].upper()
        third_codon=str(fan[old_anc].seq).replace("-","")[6:9].upper()
        if first_scodon in al_scodon or second_codon in al_scodon or third_codon in al_scodon:
            tis[last_sp] = "1"
        else:
            tis[last_sp] = "0"
    #Longest ORF in the stretch
    op = -1
    max_op[last_sp] = 0
    if len(str(fa[old_anc]).replace("-","").replace("X","*")) >= 4:
        first_scodon=str(fan[old_anc].seq).replace("-","")[0:3].upper()
        for n2 in str(fa[old_anc]).replace("-","").replace("X","*"):
            if (first_scodon in al_scodon) and (op == -1):
                op = 1
            elif n2 == "*":
                if op > max_op[last_sp]:
                    max_op[last_sp] = op
                op = -1
            elif op != -1:
                op += 1
        if op > max_op[last_sp]:
            max_op[last_sp] = op
        try:
            max_op[last_sp] = float(max_op[last_sp])/len(str(fa["hg38"]).replace("-","").replace("X","").replace("*",""))*100
        except:
            pass
    # >= 70% of the sequence did not contain stop condons truncating the ORF and a conserved ATG start condon
    if (max_op[last_sp] >= 70) and (int(tis[last_sp]) >= 1):
        max_st_cons = order[last_sp]
        ev[last_sp] = "FIXED"
    # stop condons truncating the ORF resulting in < 70% of the ORF
    else:
        ev[last_sp] = "ABSENT"
        
    max_sy_cons = order[last_sp]

# ortholog beyond intact ORF
beyond = 0
# whether is a de novo ORF
denovo = 0
# 分别代表gained gained_convergent lost
count  = [0,0,0]
for last_sp in ancestor:
    old_anc = ancestor[last_sp][0]
    print(last_sp,old_anc,order[last_sp])
    # if ancestral ortholog beyond intact ORF
    if beyond == 1:
        denovo = 1
    if (order[last_sp] == max_st_cons) or (max_st_cons == "human"):
        beyond = 1

    # Check if ORF is present in some intermediate branch
    # for each species in intermediate branch (sp2)
    for sp2 in ancestor[last_sp][1]:
        tis2 = "0"
        if len(str(fa[sp2]).replace("-","").replace("X","*")) >= 4:
            # 看是否在第一个位置或者下游的两个位置有非经典的起始密码子
            first_scodon=str(fan[sp2].seq).replace("-","")[0:3].upper()
            second_codon=str(fan[sp2].seq).replace("-","")[3:6].upper()
            third_codon=str(fan[sp2].seq).replace("-","")[6:9].upper()
            if first_scodon in al_scodon or second_codon in al_scodon or third_codon in al_scodon:
                tis2 = "1"
            else:
                tis2 = "0"

        #Longest ORF in the stretch
        op = -1
        max_op2 = 0
        for n2 in str(fa[sp2]).replace("-","").replace("X","*"):
            first_scodon=str(fan[old_anc].seq).replace("-","")[0:3].upper()
            if (first_scodon in al_scodon) and (op == -1):
                op = 1
            if n2 == "*":
                op = -1
            elif op != -1:
                op += 1
        if op > max_op2:
            max_op2 = op
        try:
            max_op2 = float(max_op2)/len(str(fa["hg38"]).replace("-","").replace("X","").replace("*",""))*100
        except:
            pass

        if (max_op2 >= 70) and (int(tis2) >= 1):
            if ev[last_sp] == "ABSENT":
                ev[last_sp] = "GAINED"
                count[0] += 1
                if beyond == 1:
                    count[1] += 1
        else:
            if ev[last_sp] == "FIXED":
                ev[last_sp] = "LOST"
                count[2] += 1

        # estimate the conservation of the ortholog peptide sequences

    #Write output
    l = name + "\t" + last_sp + "\t" + order[last_sp] + "\t" + order[last_sp] + "\t" + old_anc + "\t" + str(tis[last_sp]) + "\t" + str(max_op[last_sp]) + "\t" + ev[last_sp] + "\t" + str(fa[old_anc]).replace("-","").replace("X","*") + "\n"
    outp.write(l.replace("W\n","\n"))
    outn.write(">" + name + "--" + last_sp + "\n" + str(fan[old_anc].seq) + "\n")


outh.write(name + "\thg38\t" + str(max_st_cons) + "\t" + str(max_sy_cons) + "\t" + str(count[0]) + "\t" + str(count[1]) + "\t" + str(count[2]) + "\t" + str(denovo) + "\t" + str(fa["hg38"]).replace("-","").replace("X","") + "\n")

outn.write(">" + name + "--hg38\n" + str(fan["hg38"].seq) + "\n")

outp.close()
outn.close()
outh.close()


/home/user/BGM/lit/.local/lib/python3.9/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


{'panTro5': ['#38#', []], 'gorGor5': ['#61#', []], 'ponAbe2': ['#76#', []], 'nomLeu3': ['#84#', []], 'rheMac8': ['#92#', ['#2#', '#39#', '#3#', '#40#', '#62#', '#77#', '#4#', '#41#', '#5#', '#63#', '#85#']], 'calJac3': ['#97#', ['#6#', '#42#']], 'tarSyr2': ['#101#', []], 'otoGar3': ['#105#', ['#7#', '#43#']], 'galVar1': ['#108#', []], 'tupChi1': ['#111#', []], 'jacJac1': ['#114#', ['#8#', '#44#', '#64#', '#9#', '#45#', '#65#', '#78#', '#86#', '#93#', '#98#', '#10#', '#102#', '#11#', '#12#', '#46#', '#66#', '#106#', '#13#', '#109#', '#14#', '#112#']], 'vicPac2': ['#115#', ['#15#', '#47#', '#67#', '#16#', '#48#', '#68#', '#79#', '#17#', '#87#', '#18#', '#19#', '#49#', '#69#', '#20#', '#50#', '#70#', '#80#', '#21#', '#88#', '#94#', '#99#', '#103#', '#22#', '#51#', '#71#', '#23#', '#24#', '#52#', '#25#', '#26#', '#53#', '#27#', '#28#', '#54#', '#72#', '#81#', '#89#', '#95#', '#29#', '#100#', '#104#', '#30#', '#55#', '#31#', '#73#', '#32#', '#56#', '#74#', '#82#', '#90#', '#96#', '#107#', '

In [11]:
denovo

1

In [12]:
print(max_st_cons,max_sy_cons)

mammal mammal
